In [8]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1. Load data
# ---------------------------------------------------------
df_raw = pd.read_csv("nemsis_sprint0_3_sample.csv")

print("Raw shape:", df_raw.shape)
print(df_raw.head())

# ---------------------------------------------------------
# 2. Rename columns
# ---------------------------------------------------------
rename_map = {
    "PcrKey": "incident_id",
    "eDispatch_01": "dispatch_center_id",
    "eDispatch_02": "dispatch_call_id",
    "eResponse_05": "service_requested_type",
    "eResponse_07": "unit_transport_capability",
    "eResponse_23": "response_misc_field",
    "eScene_01": "first_unit_on_scene_flag",
    "eScene_06": "num_patients_at_scene",
    "eScene_07": "mass_casualty_flag",
    "eScene_08": "scene_factor_1",
    "eScene_09": "primary_symptom_or_scene_descr",
    "eSituation_01": "chief_complaint_location",
    "eSituation_02": "possible_injury_flag",
    "eSituation_07": "primary_impression",
    "eSituation_08": "secondary_impression",
    "eSituation_13": "trauma_score_or_severity",
    "eSituation_18": "provider_narrative_1",
    "eSituation_20": "provider_narrative_2",
    "eOutcome_01": "hospital_destination_code",
    "eOutcome_02": "ed_disposition",
    "eOutcome_11": "hospital_admit_time",
    "eOutcome_16": "hospital_discharge_time",
    "eOutcome_18": "long_term_outcome",
    "ePatient_15": "patient_age_value",
    "ePatient_16": "patient_age_units",
    "eResponse_08": "crew_size_or_delay_type",
    "eResponse_12": "response_additional_mode",
}
df = df_raw.rename(columns=rename_map)

print("\nColumns after rename:")
print(df.columns.tolist())

Raw shape: (10127, 27)
      PcrKey  eDispatch_01  eDispatch_02  eResponse_05  eResponse_07  \
0     761495       2301061       2302001       2205001       2207023   
1   22298602       2301003       2302007       2205001       2207015   
2   61958750       2301061       2302001       2205001       2207017   
3  108615464       2301061       2302001       2205001       2207015   
4  113783964       2301071       2302003       2205007       2207017   

   eResponse_23  eScene_01  eScene_06  eScene_07  eScene_08  ...  \
0       2223001    9923003    2707005    9923001    7701003  ...   
1       2223001    9923003    2707005    9923001    7701001  ...   
2       2223001    9923003    2707005    9923001    7701001  ...   
3       2223001    9923003    2707003    7701003    7701003  ...   
4       2223005    9923003    2707005    7701003    7701003  ...   

  eSituation_20 eOutcome_01  eOutcome_02           eOutcome_11  \
0    Unknown        7701003      7701003  Not Applicable         
1  

In [9]:
# ---------------------------------------------------------
# 3. Parse hospital_admit_time into datetime
#    (keep only real timestamps, drop Not Applicable/Recorded)
# ---------------------------------------------------------
raw = df["hospital_admit_time"].astype(str)

# strings that are clearly not datetimes
bad_vals = ["Not Applicable      ", "Not Recorded        ",
            "Not Applicable", "Not Recorded", ""]
bad_mask = raw.isin(bad_vals)

# candidate datetime strings
clean = raw[~bad_mask].str.strip()          # e.g. "01JAN2024:06:51:00"
# convert "01JAN2024:06:51:00" -> "01JAN2024 06:51:00"
clean = clean.str.replace(":", " ", n=1, regex=False)

parsed = pd.to_datetime(clean, format="%d%b%Y %H:%M:%S", errors="coerce")

# attach back
df["hospital_admit_time_parsed"] = pd.NaT
df.loc[~bad_mask, "hospital_admit_time_parsed"] = parsed

print("\nParse success rate:",
      df["hospital_admit_time_parsed"].notna().mean())
print(df[["hospital_admit_time", "hospital_admit_time_parsed"]]
      .head(30))

# keep only rows with a valid timestamp
df = df[df["hospital_admit_time_parsed"].notna()].copy()
df = df.sort_values("hospital_admit_time_parsed")

print("\nShape after dropping missing admit_time:", df.shape)


Parse success rate: 0.012836970474967908
     hospital_admit_time hospital_admit_time_parsed
0   Not Applicable                              NaT
1   Not Applicable                              NaT
2   Not Applicable                              NaT
3   Not Applicable                              NaT
4   Not Applicable                              NaT
5   Not Applicable                              NaT
6   Not Applicable                              NaT
7   Not Applicable                              NaT
8   Not Applicable                              NaT
9   Not Applicable                              NaT
10  Not Applicable                              NaT
11  Not Applicable                              NaT
12  Not Applicable                              NaT
13  Not Applicable                              NaT
14  Not Applicable                              NaT
15  Not Applicable                              NaT
16  Not Applicable                              NaT
17  Not Applicable    

In [10]:
# ---------------------------------------------------------
# 4. Time‑based features (from parsed admit time)
# ---------------------------------------------------------
dt = df["hospital_admit_time_parsed"]

df["admit_year"] = dt.dt.year
df["admit_month"] = dt.dt.month
df["admit_day"] = dt.dt.day
df["admit_hour"] = dt.dt.hour
df["admit_dow"] = dt.dt.dayofweek        # 0=Mon
df["admit_weekofyear"] = dt.dt.isocalendar().week.astype(int)

# hourly shift start
df["shift_start"] = dt.dt.floor("h")

print("\nTime feature example:")
print(df[["hospital_admit_time_parsed",
          "shift_start", "admit_hour",
          "admit_dow", "admit_month"]].head())


Time feature example:
     hospital_admit_time_parsed         shift_start  admit_hour  admit_dow  \
2482        2023-12-31 21:02:00 2023-12-31 21:00:00          21          6   
2819        2023-12-31 21:37:00 2023-12-31 21:00:00          21          6   
2201        2024-01-01 00:48:00 2024-01-01 00:00:00           0          0   
2651        2024-01-01 01:26:00 2024-01-01 01:00:00           1          0   
2240        2024-01-01 02:21:00 2024-01-01 02:00:00           2          0   

      admit_month  
2482           12  
2819           12  
2201            1  
2651            1  
2240            1  


In [13]:
# ---------------------------------------------------------
# 5. Age in years + pediatric / geriatric flags
# ---------------------------------------------------------
def normalize_age(row):
    val = row["patient_age_value"]
    try:
        return float(str(val).strip())
    except Exception:
        return np.nan

df["age_years"] = df.apply(normalize_age, axis=1)
df["is_pediatric"] = (df["age_years"] < 18).astype("Int8")
df["is_geriatric"] = (df["age_years"] >= 65).astype("Int8")

# scene / case-mix numeric features
df["num_patients_at_scene"] = pd.to_numeric(
    df["num_patients_at_scene"], errors="coerce"
)

# any non‑missing value in possible_injury_flag -> 1
df["possible_injury_bin"] = df["possible_injury_flag"].apply(
    lambda v: 0 if pd.isna(v) else 1
)

# optional simple trauma_high flag from trauma_score_or_severity
df["trauma_score"] = pd.to_numeric(df["trauma_score_or_severity"],
                                   errors="coerce")
if df["trauma_score"].notna().sum() > 0:
    thr = df["trauma_score"].quantile(0.75)
    df["trauma_high"] = (df["trauma_score"] >= thr).astype("Int8")
else:
    df["trauma_high"] = 0

In [14]:
# ---------------------------------------------------------
# 6. Aggregate to hourly shift level
# ---------------------------------------------------------
agg = df.groupby("shift_start").agg(
    call_volume=("incident_id", "count"),
    avg_num_patients=("num_patients_at_scene", "mean"),
    max_num_patients=("num_patients_at_scene", "max"),
    possible_injury_prop=("possible_injury_bin", "mean"),
    trauma_high_prop=("trauma_high", "mean"),
    pediatric_prop=("is_pediatric", "mean"),
    geriatric_prop=("is_geriatric", "mean"),
    service_mode_nunique=("service_requested_type", "nunique"),
).reset_index()

agg = agg.sort_values("shift_start")
agg["hour"] = agg["shift_start"].dt.hour
agg["dow"] = agg["shift_start"].dt.dayofweek
agg["month"] = agg["shift_start"].dt.month

print("\nHourly aggregation head:")
print(agg.head())


Hourly aggregation head:
          shift_start  call_volume  avg_num_patients  max_num_patients  \
0 2023-12-31 21:00:00            2         2707005.0           2707005   
1 2024-01-01 00:00:00            1         2707005.0           2707005   
2 2024-01-01 01:00:00            1         7701003.0           7701003   
3 2024-01-01 02:00:00            5         2707005.0           2707005   
4 2024-01-01 03:00:00            4         3955504.5           7701003   

   possible_injury_prop  trauma_high_prop  pediatric_prop  geriatric_prop  \
0                   1.0               1.0             0.0             1.0   
1                   1.0               1.0             0.0             0.0   
2                   1.0               1.0             1.0             0.0   
3                   1.0               0.8             0.2             0.4   
4                   1.0              0.25             0.0            0.25   

   service_mode_nunique  hour  dow  month  
0                     

In [15]:
# ---------------------------------------------------------
# 7. Define high‑strain target (top 25% call volume)
# ---------------------------------------------------------
quantile_thresh = agg["call_volume"].quantile(0.75)
agg["high_strain"] = (agg["call_volume"] >= quantile_thresh).astype(int)

print("\nHigh‑strain threshold (top 25% calls/hour):", quantile_thresh)
print(agg["high_strain"].value_counts())


High‑strain threshold (top 25% calls/hour): 5.0
high_strain
0    30
1    11
Name: count, dtype: int64


In [16]:
# ---------------------------------------------------------
# 8. Prepare features and time‑ordered train/test split
# ---------------------------------------------------------
feature_cols = [
    "hour", "dow", "month",
    "avg_num_patients", "max_num_patients",
    "possible_injury_prop", "trauma_high_prop",
    "pediatric_prop", "geriatric_prop",
    "service_mode_nunique",
]

agg[feature_cols] = agg[feature_cols].fillna(0)

X = agg[feature_cols].values
y_reg = agg["call_volume"].values
y_cls = agg["high_strain"].values

n = len(agg)
split_idx = int(n * 0.7)  # 70% earlier hours for train

X_train, X_test = X[:split_idx], X[split_idx:]
y_reg_train, y_reg_test = y_reg[:split_idx], y_reg[split_idx:]
y_cls_train, y_cls_test = y_cls[:split_idx], y_cls[split_idx:]

print("\nTrain hours:", X_train.shape[0], " Test hours:", X_test.shape[0])


Train hours: 28  Test hours: 13


In [21]:
# ---------------------------------------------------------
# 9. Models: volume regression + high‑strain classification
# ---------------------------------------------------------
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import (
    mean_absolute_error, r2_score,
    roc_auc_score, classification_report
)

# Regression: hourly call volume
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_reg_train)
y_reg_pred_lin = lin_reg.predict(X_test)

print("\n=== Linear Regression (call volume) ===")
print("MAE:", mean_absolute_error(y_reg_test, y_reg_pred_lin))
print("R^2:", r2_score(y_reg_test, y_reg_pred_lin))

rf_reg = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)
rf_reg.fit(X_train, y_reg_train)
y_reg_pred_rf = rf_reg.predict(X_test)

print("\n=== Random Forest Regressor (call volume) ===")
print("MAE:", mean_absolute_error(y_reg_test, y_reg_pred_rf))
print("R^2:", r2_score(y_reg_test, y_reg_pred_rf))

# Classification: high‑strain vs normal hour
log_cls = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)
log_cls.fit(X_train, y_cls_train)
y_prob_log = log_cls.predict_proba(X_test)[:, 1]
y_pred_log = (y_prob_log >= 0.5).astype(int)

print("\n=== Logistic Regression (high‑strain) ===")
print("ROC‑AUC:", roc_auc_score(y_cls_test, y_prob_log))
print(classification_report(y_cls_test, y_pred_log, digits=3))

rf_cls = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced_subsample"
)
rf_cls.fit(X_train, y_cls_train)
y_prob_rf = rf_cls.predict_proba(X_test)[:, 1]
y_pred_rf = (y_prob_rf >= 0.5).astype(int)

print("\n=== Random Forest Classifier (high‑strain) ===")
print("ROC‑AUC:", roc_auc_score(y_cls_test, y_prob_rf))
print(classification_report(y_cls_test, y_pred_rf, digits=3))


=== Linear Regression (call volume) ===
MAE: 1.3553833656215468
R^2: -17.30540778189656

=== Random Forest Regressor (call volume) ===
MAE: 1.038076923076923
R^2: -9.975412500000001

=== Logistic Regression (high‑strain) ===
ROC‑AUC: nan
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        13

    accuracy                          1.000        13
   macro avg      1.000     1.000     1.000        13
weighted avg      1.000     1.000     1.000        13



D:\Application\Anaconda\Lib\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(



=== Random Forest Classifier (high‑strain) ===
ROC‑AUC: nan
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        13

    accuracy                          1.000        13
   macro avg      1.000     1.000     1.000        13
weighted avg      1.000     1.000     1.000        13



D:\Application\Anaconda\Lib\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
